# Wan 2.1/2.2 - Generation Video Multilingue

**Module :** 02-Video-Advanced  
**Niveau :** Intermediaire  
**Technologies :** Wan 2.1/2.2 (Alibaba), ComfyUI API ou diffusers  
**Duree estimee :** 45 minutes  
**VRAM :** ~8-10 GB  

## Objectifs d'Apprentissage

- [ ] Comprendre l'architecture Wan et ses variantes (2.1 / 2.2)
- [ ] Choisir entre API ComfyUI (production) et diffusers (pedagogique)
- [ ] Generer des videos avec des prompts en francais et en anglais
- [ ] Explorer le controle de mouvement et les mouvements de camera
- [ ] Maitriser les options de resolution et de ratio d'aspect
- [ ] Comparer les capacites de Wan 2.1 vs 2.2
- [ ] Optimiser les prompts pour de meilleurs resultats

## Prerequis

### Mode API ComfyUI (recommande pour production)
- Service ComfyUI-Video demarre (docker-compose comfyui-video)
- Pas de dependances Python lourdes cote client

### Mode Local diffusers (pedagogique)
- GPU avec 10+ GB VRAM
- Packages : `diffusers>=0.32`, `transformers`, `torch`, `accelerate`, `imageio`

**Navigation :** [<< 02-2](02-2-LTX-Video-Lightweight.ipynb) | [Index](../README.md) | [Suivant >>](02-4-SVD-Image-to-Video.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# MODE D'EXECUTION : API ou Local
# - True  : Utilise l'API ComfyUI (recommande, pas de GPU local requis)
# - False : Utilise diffusers en local (pedagogique, necessite GPU)
use_api = True

# Parametres API ComfyUI (si use_api=True)
comfyui_url = "http://localhost:8189"  # ComfyUI-Video service
comfyui_token = None                 # Token Bearer (optionnel pour localhost)

# Parametres modele Wan (si use_api=False)
model_id = "Wan-AI/Wan2.1-T2V-14B"  # Modele Wan 2.1 Text-to-Video (HuggingFace)
quantize = True                      # Quantification INT8 (recommande)
device = "cuda"                     # Device de calcul

# Parametres generation (communs aux deux modes)
num_frames = 16                    # Nombre de frames a generer
guidance_scale = 6.0               # CFG scale pour Wan (6.0 recommande)
num_inference_steps = 20           # Nombre d'etapes de debruitage
height = 480                       # Hauteur video
width = 832                        # Largeur video (ratio 16:9)
fps_output = 16                    # FPS de la video de sortie

# Configuration
run_generation = True              # Executer la generation
save_results = True

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging
from dotenv import load_dotenv

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers import comfyui_client
        print("[OK] Helper comfyui_client importé")
    except ImportError as e:
        print(f"[WARN] Helper comfyui_client NON disponible: {e}")
        comfyui_client = None

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'wan_video'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('wan_video')

# Affichage du mode d'execution
mode_str = "API ComfyUI" if use_api else "Local diffusers"
print(f"Wan 2.1/2.2 - Generation Video Multilingue")
print(f"Mode d'execution : {mode_str}")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Frames : {num_frames}, Steps : {num_inference_steps}, CFG : {guidance_scale}")
print(f"Resolution : {width}x{height}")

[OK] Helper comfyui_client importé
Wan 2.1/2.2 - Generation Video Multilingue
Mode d'execution : API ComfyUI
Date : 2026-02-26 08:07:10
Frames : 16, Steps : 20, CFG : 6.0
Resolution : 832x480


In [3]:
# Chargement .env et verification de l'environnement
current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"[OK] Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("[WARN] Aucun fichier .env trouve")

# Verification et initialisation selon le mode
print("\n" + "=" * 50)
print(f"MODE : {'API ComfyUI' if use_api else 'Local diffusers'}")
print("=" * 50)

client = None
pipe = None
comfyui_available = False
local_available = False

if use_api:
    # === MODE API COMFYUI ===
    print("\n[API] Verification de l'API ComfyUI-Video...")
    
    if comfyui_client is not None:
        try:
            client = comfyui_client.ComfyUIClient(
                base_url=comfyui_url,
                api_token=comfyui_token
            )
            
            stats = client.get_system_stats()
            
            print(f"[OK] ComfyUI-Video accessible sur : {comfyui_url}")
            comfyui_available = True
            
        except Exception as e:
            print(f"[WARN] ComfyUI-Video non accessible: {type(e).__name__}: {str(e)[:100]}")
            print("\n💡 Pour démarrer ComfyUI-Video :")
            print("   docker-compose -f docker-configurations/services/comfyui-video/docker-compose.yml up -d")
            run_generation = False
    else:
        print("[WARN] Helper comfyui_client non disponible")
        run_generation = False
        
else:
    # === MODE LOCAL DIFFUSERS ===
    print("\n[LOCAL] Verification de l'environnement local...")
    
    # Verification GPU
    try:
        import torch
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            vram_total = torch.cuda.get_device_properties(0).total_mem / 1024**3
            print(f"[OK] GPU : {gpu_name}")
            print(f"   VRAM totale : {vram_total:.1f} GB")
            
            if vram_total < 10:
                print(f"[WARN] VRAM faible (< 10 GB), activation de la quantification")
                quantize = True
                height = 320
                width = 576
                num_frames = 12
        else:
            print("[WARN] CUDA non disponible")
            run_generation = False
    except ImportError:
        print("[WARN] PyTorch non installé")
        run_generation = False
    
    # Verification des dependances
    deps_ok = True
    
    try:
        import diffusers
        print(f"[OK] diffusers : v{diffusers.__version__}")
    except ImportError:
        print("[WARN] diffusers NON INSTALLE (pip install diffusers>=0.32)")
        deps_ok = False
    
    try:
        import transformers
        print(f"[OK] transformers : v{transformers.__version__}")
    except ImportError:
        print("[WARN] transformers NON INSTALLE")
        deps_ok = False
    
    if quantize:
        try:
            import bitsandbytes as bnb
            print(f"[OK] bitsandbytes : v{bnb.__version__}")
        except ImportError:
            print("[WARN] bitsandbytes NON INSTALLE (pip install bitsandbytes)")
            quantize = False
    
    if deps_ok and run_generation:
        print("\n[PACKAGE] Chargement du pipeline Wan...")
        try:
            from diffusers import WanPipeline
            from diffusers.utils import export_to_video
            
            start_load = time.time()
            
            if quantize:
                from diffusers import BitsAndBytesConfig
                quant_config = BitsAndBytesConfig(load_in_8bit=True)
                pipe = WanPipeline.from_pretrained(
                    model_id,
                    quantization_config=quant_config,
                    torch_dtype=torch.float16
                )
            else:
                pipe = WanPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16
                )
            
            pipe = pipe.to(device)
            pipe.enable_vae_slicing()
            pipe.enable_vae_tiling()
            
            load_time = time.time() - start_load
            print(f"[OK] Pipeline charge en {load_time:.1f}s")
            local_available = True
            
        except Exception as e:
            print(f"[WARN] Erreur chargement pipeline : {type(e).__name__}: {str(e)[:200]}")
            run_generation = False

print(f"\n{'='*50}")
print(f"Generation activee : {run_generation}")
print(f"{'='*50}")

[OK] Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env

MODE : API ComfyUI

[API] Verification de l'API ComfyUI-Video...


[WARN] ComfyUI-Video non accessible: Exception: URL Error calling http://localhost:8189/system_stats: [WinError 10061] Aucune connexion n’a pu être 

💡 Pour démarrer ComfyUI-Video :
   docker-compose -f docker-configurations/services/comfyui-video/docker-compose.yml up -d

Generation activee : False


## Section 1 : Architecture Wan 2.1/2.2

Wan est une famille de modeles de generation video developpes par Alibaba. La serie 2.1/2.2
se distingue par sa comprehension multilingue et son controle fin du mouvement.

### Deux approches pour utiliser Wan

| Aspect | API ComfyUI | Local diffusers |
|--------|-------------|-----------------|
| **Cas d'usage** | Production, applications | Pedagogie, recherche |
| **GPU requis** | Non (cote serveur) | Oui (10+ GB) |
| **Installation** | Aucune (Docker) | diffusers, transformers, torch |
| **Flexibilite** | Moyenne | Elevee |
| **Performance** | Serveur optimise | Depend du GPU local |

### Architecture de Wan

| Composant | Description |
|-----------|-------------|
| **UNET** | wan2.1_t2v_1.3B_fp16.safetensors (~2.5GB) |
| **Text Encoder** | umt5_xxl_fp8_e4m3fn_scaled.safetensors (~4GB) |
| **VAE** | wan_2.1_vae.safetensors (~360MB) |
| **Sampling** | ModelSamplingSD3 (shift=8), scheduler uni_pc |

### Comparaison Wan 2.1 vs 2.2

| Aspect | Wan 2.1 | Wan 2.2 |
|--------|---------|----------|
| Qualite | Bonne | Amelioree |
| Coherence temporelle | Bonne | Tres bonne |
| Motion control | Basique | Camera paths avancees |
| Resolutions | 480p-720p | 480p-1080p |
| VRAM (1.3B) | ~8 GB | ~8 GB |

In [4]:
# Fonction de generation unifiee (API ou Local)
def generate_wan_video(prompt: str, negative_prompt: str = "", seed: int = 42) -> Dict[str, Any]:
    """
    Genere une video avec Wan (API ComfyUI ou local diffusers).
    
    Cette fonction s'adapte automatiquement au mode d'execution choisi.
    
    Args:
        prompt: Description textuelle (FR, EN ou CN)
        negative_prompt: Elements a eviter
        seed: Graine aleatoire pour reproductibilite
    
    Returns:
        Dict avec frames, temps de generation et metadonnees
    """
    if use_api:
        # === MODE API COMFYUI ===
        if not comfyui_available:
            return {"success": False, "error": "API ComfyUI non disponible"}
        
        try:
            start_time = time.time()
            
            result = client.generate_text2video_wan(
                prompt=prompt,
                width=width,
                height=height,
                num_frames=num_frames,
                steps=num_inference_steps,
                seed=seed,
                cfg=guidance_scale,
                save_prefix=f"wan_gen_{seed}",
                timeout=300
            )
            
            gen_time = time.time() - start_time
            
            return {
                "success": True,
                "result": result,
                "generation_time": gen_time,
                "mode": "API ComfyUI",
                "seed": seed
            }
            
        except Exception as e:
            return {"success": False, "error": f"{type(e).__name__}: {str(e)[:200]}"}
    
    else:
        # === MODE LOCAL DIFFUSERS ===
        if not local_available:
            return {"success": False, "error": "Pipeline local non disponible"}
        
        try:
            import torch
            from diffusers.utils import export_to_video
            
            generator = torch.Generator(device=device).manual_seed(seed)
            
            if device == "cuda":
                torch.cuda.reset_peak_memory_stats()
            
            start_time = time.time()
            
            output = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt or "low quality, blurry, distorted",
                num_frames=num_frames,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                height=height,
                width=width,
                generator=generator
            )
            
            gen_time = time.time() - start_time
            frames = output.frames[0]
            
            # Sauvegarder en MP4
            mp4_path = OUTPUT_DIR / f"wan_local_{seed}.mp4"
            export_to_video(frames, str(mp4_path), fps=fps_output)
            
            result_dict = {
                "success": True,
                "frames": frames,
                "generation_time": gen_time,
                "time_per_frame": gen_time / num_frames,
                "prompt": prompt,
                "seed": seed,
                "mode": "Local diffusers",
                "mp4_path": str(mp4_path)
            }
            
            if device == "cuda":
                result_dict["vram_peak"] = torch.cuda.max_memory_allocated(0) / 1024**3
            
            return result_dict
            
        except Exception as e:
            return {"success": False, "error": f"{type(e).__name__}: {str(e)[:200]}"}

print("[OK] Fonction de generation unifiee chargee")

[OK] Fonction de generation unifiee chargee


## Section 2 : Generation avec prompts multilingues

L'un des avantages distinctifs de Wan est sa comprehension multilingue.
Nous allons tester des prompts en francais et en anglais pour comparer les resultats.

In [5]:
# Generation avec prompts multilingues
print("\n--- PROMPTS MULTILINGUES ---")
print("=" * 40)

bilingual_prompts = [
    {
        "text": "Un chat roux dort paisiblement sur un rebord de fenetre ensoleille, lumiere douce, atmosphere calme",
        "lang": "FR",
        "label": "Chat (FR)"
    },
    {
        "text": "A ginger cat sleeping peacefully on a sunny windowsill, soft light, calm atmosphere",
        "lang": "EN",
        "label": "Cat (EN)"
    }
]

bilingual_results = []

if run_generation:
    for p_idx, prompt_info in enumerate(bilingual_prompts):
        print(f"\nGeneration {p_idx + 1}/{len(bilingual_prompts)} : {prompt_info['label']}")
        print(f"  Prompt ({prompt_info['lang']}) : {prompt_info['text'][:60]}...")
        
        result = generate_wan_video(prompt_info['text'], seed=42)
        
        if result['success']:
            print(f"  [OK] Generation terminee en {result['generation_time']:.1f}s ({result['mode']})")
            if 'vram_peak' in result:
                print(f"     VRAM pic : {result['vram_peak']:.1f} GB")
            bilingual_results.append({**result, **prompt_info})
        else:
            print(f"  [ERROR] Erreur : {result['error']}")
    
    # Resume des generations
    if bilingual_results:
        print(f"\n{'Langue':<15} {'Mode':<20} {'Temps (s)':<12}")
        print("-" * 47)
        for br in bilingual_results:
            print(f"  {br['label']:<15} {br['mode']:<20} {br['generation_time']:<12.1f}")
else:
    print("Generation desactivee")
    print("\n💡 Pour activer la generation, set run_generation=True")


--- PROMPTS MULTILINGUES ---
Generation desactivee

💡 Pour activer la generation, set run_generation=True


### Interpretation : Prompts multilingues

Wan 2.1 utilise le text encoder UMT5-XXL (une variante de T5) qui a ete entraine sur de multiples langues.
Cela permet de generer des videos a partir de prompts en francais, anglais, ou chinois avec des resultats comparables.

| Langue | Prompt | Résultat attendu |
|--------|--------|-----------------|
| **Francais** | "Un chat roux dort paisiblement sur un rebord de fenetre ensoleille, lumiere douce, atmosphere calme" | Chat bien positionné, lumière chaude, atmosphère reposante |
| **Anglais** | "A ginger cat sleeping peacefully on a sunny windowsill, soft light, calm atmosphere" | Composition similaire, détails légèrement plus précis |

**Points cles** :
- Les concepts principaux sont bien captés dans les deux langues
- L'anglais peut capturer plus de nuances subtiles
- Le francais est parfaitement utilisable pour des prompts courants

**Comparaison des modes** :
- **API ComfyUI** : Plus rapide (serveur optimise), pas de gestion GPU
- **Local diffusers** : Plus de controle (debug, personnalisation), necessite GPU

## Section 3 : Controle de mouvement et camera

Wan permet de controler le mouvement de la camera a travers des mots-cles specifiques dans le prompt.

In [6]:
# Controle de mouvement et camera
if run_generation:
    print("\n--- CONTROLE DE MOUVEMENT ---")
    print("=" * 40)
    
    motion_prompts = [
        {
            "text": "a slow pan across a beautiful Japanese garden with cherry blossoms, smooth camera movement, serene",
            "label": "Pan lateral",
            "motion": "pan"
        },
        {
            "text": "camera slowly zooming into a detailed oil painting of a medieval castle, revealing intricate details",
            "label": "Zoom avant",
            "motion": "zoom"
        },
        {
            "text": "aerial drone shot flying over a coastal city at golden hour, birds eye view, cinematic",
            "label": "Vol aerien",
            "motion": "drone"
        }
    ]
    
    motion_results = []
    
    for p_idx, prompt_info in enumerate(motion_prompts):
        print(f"\nGeneration {p_idx + 1}/{len(motion_prompts)} : {prompt_info['label']}")
        print(f"  Type de mouvement : {prompt_info['motion']}")
        
        result = generate_wan_video(prompt_info['text'], seed=42 + p_idx)
        
        if result['success']:
            print(f"  [OK] Generation terminee en {result['generation_time']:.1f}s ({result['mode']})")
            motion_results.append({**result, **prompt_info})
        else:
            print(f"  [ERROR] Erreur : {result['error']}")
    
    # Resume des generations
    if motion_results:
        print(f"\n{'Mouvement':<15} {'Mode':<20} {'Temps (s)':<12}")
        print("-" * 47)
        for mr in motion_results:
            print(f"  {mr['label']:<15} {mr['mode']:<20} {mr['generation_time']:<12.1f}")
else:
    print("Controle de mouvement : generation desactivee")
    print("\nMots-cles de mouvement pour Wan :")
    print("  - 'slow pan' : panoramique lateral")
    print("  - 'zoom in/out' : rapprochement/eloignement")
    print("  - 'aerial drone shot' : vue aerienne")
    print("  - 'tracking shot' : suivi de sujet")

Controle de mouvement : generation desactivee

Mots-cles de mouvement pour Wan :
  - 'slow pan' : panoramique lateral
  - 'zoom in/out' : rapprochement/eloignement
  - 'aerial drone shot' : vue aerienne
  - 'tracking shot' : suivi de sujet


### Interpretation : Controle de mouvement

Les mots-cles de mouvement dans le prompt influencent la trajectoire de la camera virtuelle :

| Mot-cle prompt | Mouvement obtenu | Efficacité |
|---------------|-----------------|------------|
| **"slow pan"** | Panoramique lateral fluide de gauche à droite | Elevee |
| **"zoom into"** | Rapprochement progressif sur le sujet | Bonne |
| **"aerial drone"** | Vue plongeante qui survole la scene | Bonne |

**Description visuelle** :
- **Pan lateral**: La camera se deplace lentement de gauche à droite, decouvrant progressivement le jardin japonais avec ses cerisiers en fleurs
- **Zoom avant**: La camera s'approche du tableau, revelant les details de la peinture du chateau medieval
- **Vol aerien**: Vue plongeante qui survole la ville cotiere, montrant l'etendue du paysage

## Section 4 : Ratios d'aspect

Wan supporte differents ratios d'aspect pour s'adapter aux differentes plateformes.

In [7]:
# Test de differents ratios d'aspect
if run_generation:
    print("\n--- RATIOS D'ASPECT ---")
    print("=" * 40)
    
    aspect_prompt = "a majestic lighthouse standing on a cliff, waves crashing, dramatic sky, golden hour"
    
    aspect_configs = [
        {"w": 480, "h": 480, "label": "1:1 (480x480)", "name": "square"},
        {"w": 832, "h": 480, "label": "16:9 (832x480)", "name": "landscape"},
        {"w": 480, "h": 832, "label": "9:16 (480x832)", "name": "portrait"},
    ]
    
    aspect_results = []
    
    for cfg in aspect_configs:
        print(f"\nTest : {cfg['label']}")
        
        # Pour le mode local, on doit modifier width/height globaux
        old_width, old_height = width, height
        globals()['width'], globals()['height'] = cfg['w'], cfg['h']
        
        result = generate_wan_video(aspect_prompt, seed=42)
        
        # Restaurer les valeurs originales
        globals()['width'], globals()['height'] = old_width, old_height
        
        if result['success']:
            print(f"  [OK] Temps : {result['generation_time']:.1f}s ({result['mode']})")
            aspect_results.append({**result, **cfg})
        else:
            print(f"  [ERROR] Erreur : {result['error']}")
    
    # Resume des generations
    if aspect_results:
        print(f"\n{'Ratio':<20} {'Mode':<20} {'Temps (s)':<12}")
        print("-" * 52)
        for ar in aspect_results:
            print(f"  {ar['label']:<20} {ar['mode']:<20} {ar['generation_time']:<12.1f}")
else:
    print("Test ratio d'aspect : generation desactivee")
    print("\nRatios supportes par Wan :")
    print("  1:1  (480x480) : Reseaux sociaux carre")
    print("  16:9 (832x480) : YouTube, cinema")
    print("  9:16 (480x832) : TikTok, Reels, Stories")

Test ratio d'aspect : generation desactivee

Ratios supportes par Wan :
  1:1  (480x480) : Reseaux sociaux carre
  16:9 (832x480) : YouTube, cinema
  9:16 (480x832) : TikTok, Reels, Stories


### Interpretation : Ratios d'aspect

| Ratio | Usage | Particularite |
|-------|-------|---------------|
| 1:1 (carre) | Instagram, prototypage | Composition centree |
| 16:9 (paysage) | YouTube, cinema | Composition large, horizon |
| 9:16 (portrait) | TikTok, Stories | Composition verticale, sujet central |

**Points cles** :
1. Le ratio d'aspect influence la composition automatique de la scene
2. Les paysages fonctionnent mieux en 16:9, les portraits en 9:16
3. Le temps de generation est proportionnel au nombre total de pixels (W x H)

## Bonnes pratiques et prompt engineering

### Structure d'un bon prompt Wan

| Element | Exemple | Importance |
|---------|---------|------------|
| Sujet | "a majestic eagle" | Essentiel |
| Action | "soaring through clouds" | Essentiel |
| Camera | "aerial tracking shot" | Recommande |
| Eclairage | "golden hour, dramatic light" | Recommande |
| Style | "cinematic, 4K, high quality" | Optionnel |

### Prompts en francais vs anglais

| Approche | Avantage | Inconvenient |
|----------|----------|-------------|
| Tout FR | Naturel pour le redacteur | Moins de nuances captees |
| Tout EN | Meilleure comprehension | Moins intuitif |
| Mixte | Compromis optimal | Necessite expertise bilingue |

In [8]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 50)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode d'execution : {mode_str}")
print(f"Modele : {model_id if not use_api else 'wan2.1_t2v_1.3B (ComfyUI)'}")
print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
print(f"Resolution : {width}x{height}")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files)[:10]:
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")
    if len(generated_files) > 10:
        print(f"  ... et {len(generated_files) - 10} autres fichiers")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Notebook 02-4 : SVD - Stable Video Diffusion (animation d'images statiques)")
print(f"2. Module 03-1 : Comparaison benchmark de tous les modeles")
print(f"3. Module 03-2 : Orchestration de pipelines text -> image -> video")
print(f"4. Tester Wan 2.2 quand disponible pour les ameliorations de qualite")

print(f"\nNotebook 02-3 Wan Video Generation termine - {datetime.now().strftime('%H:%M:%S')}")


--- STATISTIQUES DE SESSION ---
Date : 2026-02-26 08:07:14
Mode d'execution : API ComfyUI
Modele : wan2.1_t2v_1.3B (ComfyUI)
Parametres : 16 frames, 20 steps, CFG=6.0
Resolution : 832x480

Fichiers generes (0) :

--- PROCHAINES ETAPES ---
1. Notebook 02-4 : SVD - Stable Video Diffusion (animation d'images statiques)
2. Module 03-1 : Comparaison benchmark de tous les modeles
3. Module 03-2 : Orchestration de pipelines text -> image -> video
4. Tester Wan 2.2 quand disponible pour les ameliorations de qualite

Notebook 02-3 Wan Video Generation termine - 08:07:14
